In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.optimizers import SGD

dataName = "KU-HAR"

input_nc = 6
segment_size = 300
class_num = 8 # initial_classes

data_path = "../Data/" + dataName + "/deploy"
train_X = np.load(data_path+'/train_x0.npy')
train_Y = np.load(data_path+'/train_y0.npy')[:,:8]
test_X = np.load(data_path+'/test_x0.npy')
test_Y = np.load(data_path+'/test_y0.npy')[:,:8]


train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_Y))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)



class Model(tf.keras.Model):

  def __init__(self):
    super(Model, self).__init__()
    self.backbone = tf.keras.Sequential([
        tf.keras.layers.Conv1D(64, 3, strides=1, padding='SAME', activation='relu', input_shape=(segment_size, input_nc)),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(128, 3, strides=1, padding='SAME', activation='relu'),
        tf.keras.layers.Conv1D(128, 3, strides=1, padding='SAME', activation='relu'),
        tf.keras.layers.GlobalAveragePooling1D(data_format='channels_last'),
    ])

    self.fc = tf.keras.layers.Dense(class_num, name='dense_1')

  def call(self, x, training=True):
    features = self.backbone(x)
    return self.fc(features)
  
NUM_EPOCHS = 300
epochs = np.arange(1, NUM_EPOCHS + 1, 1)
losses = np.zeros([NUM_EPOCHS])
m = Model()

train_ds = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
train_ds = train_ds.batch(BATCH_SIZE)

opt = SGD(lr=0.01)
m.compile(
  optimizer = opt,
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy']
)

def decay(epoch):
    if epoch < 150:
      return 0.1
    if epoch >= 150 and epoch < 225:
      return 0.01
    if epoch >= 225:
      return 0.001

m.fit(train_dataset, epochs=300, validation_data=test_dataset,
        verbose=2, callbacks=[tf.keras.callbacks.LearningRateScheduler(
            decay)])

m.save("pretrained")

Epoch 1/300


d:\codes\PHAR\.venv\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


181/181 - 1s - loss: 1.4571 - accuracy: 0.4363 - val_loss: 0.9763 - val_accuracy: 0.6165 - lr: 0.1000 - 1s/epoch - 6ms/step
Epoch 2/300
181/181 - 1s - loss: 0.8935 - accuracy: 0.5917 - val_loss: 0.8022 - val_accuracy: 0.6404 - lr: 0.1000 - 502ms/epoch - 3ms/step
Epoch 3/300
181/181 - 0s - loss: 0.8014 - accuracy: 0.6366 - val_loss: 0.7564 - val_accuracy: 0.6793 - lr: 0.1000 - 485ms/epoch - 3ms/step
Epoch 4/300
181/181 - 0s - loss: 0.7665 - accuracy: 0.6490 - val_loss: 0.8284 - val_accuracy: 0.6111 - lr: 0.1000 - 484ms/epoch - 3ms/step
Epoch 5/300
181/181 - 0s - loss: 0.7455 - accuracy: 0.6615 - val_loss: 0.6824 - val_accuracy: 0.7079 - lr: 0.1000 - 482ms/epoch - 3ms/step
Epoch 6/300
181/181 - 0s - loss: 0.7121 - accuracy: 0.6754 - val_loss: 0.7249 - val_accuracy: 0.6789 - lr: 0.1000 - 493ms/epoch - 3ms/step
Epoch 7/300
181/181 - 0s - loss: 0.7056 - accuracy: 0.6874 - val_loss: 0.6656 - val_accuracy: 0.6725 - lr: 0.1000 - 483ms/epoch - 3ms/step
Epoch 8/300
181/181 - 0s - loss: 0.7035 - 

INFO:tensorflow:Assets written to: pretrained\assets


INFO:tensorflow:Assets written to: pretrained\assets
